# Auto Testbench
* use 200 images that are only for testing purpose
* automatic testing and result output

### 1. Importing libraries

In [3]:
import os
import shutil
import numpy as np
import matplotlib.pyplot as plt
import cv2
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image

import sys
sys.path.append("../")
from Constants import Constants

### 2. Defining constants
this is the part where you can change the constants to your need

In [4]:
# Constants
IMAGE_SIZE = Constants.CNN_IMAGE_SIZE
ROI_COORDINATES = Constants.ROI_COORDINATES
CATEGORY_DICT = Constants.CATEGORY_DICT
TEST_CATEGORIES = Constants.TEST_CATEGORIES

model_dir = Constants.MODEL_PATH
model_name = Constants.WLI_MODEL_NAME
test_dir = Constants.PATH_WLI_TESTSET
failure_dir = Constants.PATH_WLI_FAILED
preprocessing_buffer_dir = Constants.PATH_WLI_BUFFER

### 3. Utility functions
generally do not need to change anything here

In [5]:
def check_test_directories(test_dir, test_categories):
    for category in test_categories:
        path = os.path.join(test_dir, category)
        if os.path.exists(path):
            print("Path exists: " + path)
            print(category + " has " + str(len(os.listdir(path))) + " files.")
        else:
            print("Path does not exist: " + path)

def process_image(img_path, img_size, roi_coordinates):
    img = cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    # Skip processing if the image is already of the right size
    if img.shape[:2] == img_size[::-1]:  # cv2 image size is in (height, width) format
        return np.expand_dims(img.astype('float32') / 255, axis=0)

    x_start, x_end, y_start, y_end = roi_coordinates
    img_roi = img[y_start:y_end, x_start:x_end]
    img = cv2.resize(img_roi, img_size)
    return np.expand_dims(img.astype('float32') / 255, axis=0)


def test_single_image(model, img_path):
    img = process_image(img_path, IMAGE_SIZE, ROI_COORDINATES)
    prediction = model.predict(img)
    class_ = np.argmax(prediction, axis=1)
    return CATEGORY_DICT[class_[0]]

def test_multiple_images(model, test_dir, test_categories, preprocessing_buffer_dir, print_info):
    success_dict = {}
    failure_dict = {}
    
    for cat in test_categories:
        img_files = os.listdir(os.path.join(test_dir, cat))
        for fname in img_files:
            img_path = os.path.join(test_dir, cat, fname)
            img = process_image(img_path, IMAGE_SIZE, ROI_COORDINATES)
            prediction = model.predict(img)
            class_ = CATEGORY_DICT[np.argmax(prediction, axis=1)[0]]

            if cat == class_:
                success_dict[fname] = class_
            else:
                failure_dict[fname] = class_

    if print_info:
        print("Success: ", success_dict)
        print("Failure: ", failure_dict)    
        print("Success rate: ", len(success_dict)/(len(success_dict)+len(failure_dict)))
    
    return success_dict, failure_dict

def copy_failure_files(failure_dict, test_dir, failure_dir, test_categories):
    for cat in test_categories:
        os.makedirs(os.path.join(failure_dir, cat), exist_ok=True)
        for file in failure_dict.keys():
            if os.path.exists(os.path.join(test_dir, cat, file)):
                shutil.copy(os.path.join(test_dir, cat, file), os.path.join(failure_dir, cat, file))

### 4. Driver codes
do not need to change anything here, just run the cell

In [6]:
# Driver code 
model = load_model(os.path.join(model_dir, model_name))
success_dict, failure_dict = test_multiple_images(model, test_dir, TEST_CATEGORIES, preprocessing_buffer_dir, False)
print("Success: ", success_dict)
print("Failure: ", failure_dict)
print("Success rate: ", len(success_dict)/(len(success_dict)+len(failure_dict)))
copy_failure_files(failure_dict, test_dir, failure_dir, TEST_CATEGORIES)

1/1 [==============================] - 0s 164ms/step
Success:  {'OverSaturatedClose0.bmp': 'OverSaturated', 'OverSaturatedClose1.bmp': 'OverSaturated', 'OverSaturatedClose2.bmp': 'OverSaturated', 'OverSaturatedClose3.bmp': 'OverSaturated', 'OverSaturatedClose4.bmp': 'OverSaturated', 'OverSaturatedClose5.bmp': 'OverSaturated', 'OverSaturatedClose6.bmp': 'OverSaturated', 'OverSaturatedClose7.bmp': 'OverSaturated', 'OverSaturatedClose8.bmp': 'OverSaturated', 'OverSaturatedClose9.bmp': 'OverSaturated', 'OverSaturatedExClose0.bmp': 'OverSaturated', 'OverSaturatedExClose1.bmp': 'OverSaturated', 'OverSaturatedExClose2.bmp': 'OverSaturated', 'OverSaturatedExClose3.bmp': 'OverSaturated', 'OverSaturatedExClose4.bmp': 'OverSaturated', 'OverSaturatedExClose5.bmp': 'OverSaturated', 'OverSaturatedExClose6.bmp': 'OverSaturated', 'OverSaturatedExClose7.bmp': 'OverSaturated', 'OverSaturatedExClose8.bmp': 'OverSaturated', 'OverSaturatedExClose9.bmp': 'OverSaturated', 'OverSaturatedFar0.bmp': 'OverSatura